## ASIGNACION DE VIAJE

In [ ]:
# Import Python Libraries
import logging
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.transforms import CoGroupByKey
import json

# Definir suscripciones y otros detalles si es necesario
suscripcion_coche = 'projects/dataflow-1-411618/subscriptions/coches_stream-sub'
suscripcion_usuario = 'projects/dataflow-1-411618/subscriptions/usuarios_stream-sub'

# Recibe datos
class DecodeMessage(beam.DoFn):
    def process(self, element):
        output = element.decode('utf-8')
        json_data = json.loads(output)
        return [json_data]

# Función para extraer la clave 'geo' de cada elemento
def extract_geo(element):
    geo = element.get('geo', None)
    return (geo, element)

# Función para filtrar casos coincidentes y no coincidentes
class FilterCoincidentCases(beam.DoFn):
    def process(self, element):
        geo_key, messages = element
        coches = messages['coches']
        usuarios = messages['usuarios']

        if coches and usuarios:
            yield {'geo': geo_key, 'coches': coches, 'usuarios': usuarios, 'coincidente': True}
        else:
            yield {'geo': geo_key, 'coches': coches, 'usuarios': usuarios, 'coincidente': False}

# Crear el pipeline
with beam.Pipeline(options=PipelineOptions(streaming=True)) as p:
    # Coches
    coches_data = (
        p
        | "Coche_LeerDesdePubSub" >> beam.io.ReadFromPubSub(subscription=suscripcion_coche)
        | "Coche_decodificar_msg" >> beam.ParDo(DecodeMessage())
        | "Coche_Extraer_Clave_geo" >> beam.Map(extract_geo)
        | "Coche_ventana_5_minutos" >> beam.WindowInto(beam.window.FixedWindows(50))
    )

    # Usuarios
    usuarios_data = (
        p
        | "Usuario_LeerDesdePubSub" >> beam.io.ReadFromPubSub(subscription=suscripcion_usuario)
        | "Usuario_decodificar_msg" >> beam.ParDo(DecodeMessage())
        | "Usuario_Extraer_Clave_geo" >> beam.Map(extract_geo)
        | "Usuario_ventana_5_minutos" >> beam.WindowInto(beam.window.FixedWindows(50))
    )

    # Realizar un CoGroupByKey en base al campo 'geo'
    joined_data = (
        {'coches': coches_data, 'usuarios': usuarios_data}
        | "Merge_Mensajes_por_geo" >> CoGroupByKey()
        | "Filtrar_Casos_Coincidentes" >> beam.ParDo(FilterCoincidentCases())
        | "Filtrar_Solo_Coincidentes" >> beam.Filter(lambda element: element['coincidente'])
        | "Imprimir_Resultados" >> beam.Map(lambda element: print('Inicio de viaje:', element))
    )

Inicio de viaje: {'geo': '39.49597-0.38838', 'coches': [{'coche_id_message': 'dc20670c-e3b0-4f5c-a50d-eb1b84f0e18a', 'coche_id': 1, 'coche_index_msg': 11, 'geo': '39.49597-0.38838', 'coche_latitud': 39.49597, 'coche_longitud': -0.38838, 'datetime': '2024-01-29T20:53:21.071263Z', 'coche_ruta': 'benicalap-alboraya.kml'}], 'usuarios': [{'user_id_message': '434e2471-9853-4bca-bc68-78596524d39c', 'user_id': 9990, 'datetime': '2024-01-29T20:52:59.792564Z', 'geo': '39.49597-0.38838', 'user_geo_fin': '39.49746-0.38488', 'user_latitud_inicio': 39.49597, 'user_longitud_inicio': -0.38838, 'user_latitud_destino': 39.49746, 'user_longitud_destino': -0.38488}], 'coincidente': True}


### ASIGNACION VIAJE + FIN

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.transforms import CoGroupByKey
import json

################config################################

options = PipelineOptions(
    streaming=True,
    runner='DataflowRunner',
    experiments='enable_streaming_engine,use_beam_bq_sink'
)

# Definir suscripciones y otros detalles si es necesario
suscripcion_coche = 'projects/dataflow-1-411618/subscriptions/coches_stream-sub'
suscripcion_usuario = 'projects/dataflow-1-411618/subscriptions/usuarios_stream-sub'

#########funciones#######################################

# Recibe datos
class DecodeMessage(beam.DoFn):
    def process(self, element):
        output = element.decode('utf-8')
        json_data = json.loads(output)
        return [json_data]

# Función para extraer la clave 'user_geo_fin' de cada elemento
def extract_geo_fin(element):
    geo = element.get('user_geo_fin', None)
 #   print(geo)
    return (geo, element)

# Función para extraer la clave 'geo' de cada elemento
def extract_geo(element):
    geo = element.get('geo', None)
 #   print(geo)
    return (geo, element)


# Función para filtrar casos coincidentes y no coincidentes
class FilterCoincidentCases(beam.DoFn):
    def process(self, element):
        geo_key, messages = element
        coches = messages['coches']
        usuarios = messages['usuarios']

        if coches and usuarios:
            yield {'geo': geo_key, 'coches': coches, 'usuarios': usuarios, 'coincidente': True}
        else:
            yield {'geo': geo_key, 'coches': coches, 'usuarios': usuarios, 'coincidente': False}

            
##########pipeline#################################            
            
# Crear el pipeline
with beam.Pipeline(options=PipelineOptions(streaming=True, save_main_session=True)) as p:
    # Coches
    coches_data = (
        p
        | "Coche_LeerDesdePubSub" >> beam.io.ReadFromPubSub(subscription=suscripcion_coche)
        | "Coche_decodificar_msg" >> beam.ParDo(DecodeMessage())
        | "Coche_Extraer_Clave_geo" >> beam.Map(extract_geo)
        | "Coche_ventana_5_minutos" >> beam.WindowInto(beam.window.FixedWindows(10))
    )

    # Usuarios_inicio
    usuarios_data = (
        p
        | "Usuario_LeerDesdePubSub" >> beam.io.ReadFromPubSub(subscription=suscripcion_usuario)
        | "Usuario_decodificar_msg" >> beam.ParDo(DecodeMessage())
        | "Usuario_Extraer_Clave_geo" >> beam.Map(extract_geo)
        | "Usuario_ventana_5_minutos" >> beam.WindowInto(beam.window.FixedWindows(10))
    )

    
    # Usuarios_fin
    usuarios_data_fin = (
        p
        | "Usuario_LeerDesdePubSub_inic" >> beam.io.ReadFromPubSub(subscription=suscripcion_usuario)
        | "Usuario_decodificar_msg_inic" >> beam.ParDo(DecodeMessage())
        | "Usuario_Extraer_Clave_geo_inic" >> beam.Map(extract_geo_fin)
        | "Usuario_ventana_5_minutos_inic" >> beam.WindowInto(beam.window.FixedWindows(10))
    )
    
    # Realizar un CoGroupByKey en base al campo 'geo'_inicio
    joined_data_inicio = (
        {'coches': coches_data, 'usuarios': usuarios_data}
        | "Merge_Mensajes_por_geo" >> CoGroupByKey()
        | "Filtrar_Casos_Coincidentes" >> beam.ParDo(FilterCoincidentCases())
        | "Filtrar_Solo_Coincidentes" >> beam.Filter(lambda element: element['coincidente'])
        | "Imprimir_Resultados_inic" >> beam.Map(lambda element: print('Inicio de viaje:', element))
    )

    
    # Realizar un CoGroupByKey en base al campo 'geo'_fin
    joined_data_fin = (
        {'coches': coches_data, 'usuarios': usuarios_data_fin}
        | "Merge_Mensajes_por_geo_fin" >> CoGroupByKey()
        | "Filtrar_Casos_Coincidentes_fin" >> beam.ParDo(FilterCoincidentCases())
        | "Filtrar_Solo_Coincidentes_fin" >> beam.Filter(lambda element: element['coincidente'])
        | "Imprimir_Resultados_fin" >> beam.Map(lambda element: print('Fin de viaje:', element))
    )

### Read PubSub - Write Bigquery

In [3]:

################################ Script para escribir en Big Query la información de los coches ####################################

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.io.gcp.bigquery import WriteToBigQuery
from apache_beam.io.gcp.internal.clients import bigquery
import json
from datetime import datetime

#################################################### Cris ######################################################
project_id = 'dataflow-1-411618'
topic_name= 'clase_test'
table_name = 'dataflow-1-411618:blablacar.coches'
suscripcion = 'projects/dataflow-1-411618/subscriptions/coches_stream-sub'
################################################################################################################



# Recibe datos
def decode_message(msg):
    # Lógica para decodificar el mensaje y cargarlo como JSON
    output = msg.decode('utf-8')
    json_data = json.loads(output)
    logging.info("New message in PubSub: %s", json_data)
    return json_data

# Obtiene la hora actual en formato UTC
current_time_utc = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'

class DecodeMessage(beam.DoFn):
    def process(self, element):
        output = element.decode('utf-8')
        json_data = json.loads(output)
        print(f"JSON guardado en BigQuery: {json_data}")
        return [json_data]

# Nueva definición del esquema para BigQuery
new_table_schema = bigquery.TableSchema()
new_table_fields = [
    bigquery.TableFieldSchema(name='id_message', type='STRING', mode='NULLABLE'),
    bigquery.TableFieldSchema(name='coche_id', type='INT', mode='NULLABLE'),
    bigquery.TableFieldSchema(name='index_msg', type='STRING', mode='NULLABLE'),
    bigquery.TableFieldSchema(name='latitud', type='FLOAT', mode='NULLABLE'),
    bigquery.TableFieldSchema(name='longitud', type='FLOAT', mode='NULLABLE'),
    bigquery.TableFieldSchema(name='datetime', type='DATETIME', mode='NULLABLE'),
    bigquery.TableFieldSchema(name='ruta', type='STRING', mode='NULLABLE')
]
new_table_schema.fields.extend(new_table_fields)





with beam.Pipeline(options=PipelineOptions(streaming=True)) as p:
    #coches:
    data = (
        p
        | "LeerDesdePubSub" >> beam.io.ReadFromPubSub(subscription=suscripcion)
        | "decodificar_msg" >> beam.ParDo(DecodeMessage())
        | "escribir" >> beam.io.WriteToBigQuery(
            table= table_name,
            schema=new_table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_NEVER,
            write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
        )
    )



JSON guardado en BigQuery: {'coche_id_message': None, 'coche_id': 1, 'cocher_index_msg': 1, 'cocher_geo': '39.49597-0.38838', 'cocher_latitud': 39.49597, 'coche_longitud': -0.38838, 'coche_datetime': None, 'coche_ruta': 'benicalap-alboraya.kml', 'datetime': '2024-01-28T11:33:13.026913', 'id_message': '070a14c8-4415-4370-8677-731007a33527'}


JSON guardado en BigQuery: {'coche_id_message': None, 'coche_id': 1, 'cocher_index_msg': 2, 'cocher_geo': '39.49658-0.38742', 'cocher_latitud': 39.49658, 'coche_longitud': -0.38742, 'coche_datetime': None, 'coche_ruta': 'benicalap-alboraya.kml', 'datetime': '2024-01-28T11:33:14.031303', 'id_message': '1d831d96-8cf3-4448-a493-a6f2259b22db'}


KeyboardInterrupt: 